## Логистическая регрессия в задаче классификации текстов

### Описание задачи

Реализована модель multilabel классификации текстов, использующая онлайн обучение (стохастический градиентный спуск с логистической функцией потерь). В качестве примера взят датасет вопросов на stackoverflow, задачей стоит предсказание тегов вопросов.

Данные доступны для скачивания по ссылке: https://yadi.sk/d/Vvc4JmX13ZdWvk

За основу было взято задание курса OpenDataScience. Автор задания - Павел Нестеров (@mephistopheies).

### Формат входных данных

Входные данные представляют собой файл с вопросами пользователей stackoverflow.com. В каждой строке файла содержится текст вопроса и список тегов, разделенные символом табуляции. Теги в списке разделены пробелом.

### Описание математической основы

Рассмотрим логистическую регрессию для двух классов $\{0, 1\}$. Обозначим вектор признаков объекта как $\textbf{x}$. Вероятность принадлежности объекта классу $1$, вспомнив теорему Байеса, можно записать как:

$$
p\left(c = 1 \mid \textbf{x}\right) = 
\dfrac
{p\left(\textbf{x} \mid c = 1\right)p\left(c = 1\right)}
{p\left(\textbf{x}\right)}
$$

Воспользуясь формулой полной вероятности, получаем:

$$
p\left(c = 1 \mid \textbf{x}\right) = 
\dfrac
{p\left(\textbf{x} \mid c = 1\right)p\left(c = 1\right)}
{p\left(\textbf{x} \mid c = 1\right)p\left(c = 1\right) + p\left(\textbf{x} \mid c = 0\right)p\left(c = 0\right)}
$$

Введя параметр:

$$
a = \log \dfrac
{p\left(\textbf{x} \mid c = 1\right)p\left(c = 1\right)}
{p\left(\textbf{x} \mid c = 0\right)p\left(c = 0\right)}
$$

Мы можем наше выражение переписать в виде:

$$
p\left(c = 1 \mid \textbf{x}\right) = \dfrac{1}{1 + e^{-a}} = \sigma\left(a\right)
$$

где $\sigma\left(a\right)$ - обозначение функции логистического сигмоида для скалярного аргумента.

Значение же параметра $a$ мы моделируем линейной функцией от признаков объекта и параметров модели:

$$
a = \sum_{i=0}^M w_i x_i
$$

---

Обобщим подход до задачи многоклассовой классификации, то есть у нас есть $K$ классов, к которым может принадлежать объект: $\{1, 2, ..., K\}$. Запишем вероятность принадлежности объекта классу $k$:

$$
p\left(c = k \mid \textbf{x}\right) = 
\dfrac
{p\left(\textbf{x} \mid c = k\right)p\left(c = k\right)}
{p\left(\textbf{x}\right)} =
\dfrac
{p\left(\textbf{x} \mid c = k\right)p\left(c = k\right)}
{\sum_{i=1}^Kp\left(\textbf{x} \mid c = i\right)p\left(c = i\right)}
$$

Введем параметр:

$$
z_k = \log p\left(\textbf{x} \mid c=k \right) p\left(c = k\right)
$$

Перепишем наше выражение в виде:

$$
p\left(c = k \mid \textbf{x}\right) =
\dfrac
{e^{z_k}}
{\sum_{i=1}^K e^{z_i}}
= \sigma_k(\textbf{z})
$$

где $\sigma_k$ - $k$-ый компонент функции softmax (обобщение логистической регрессии для многомерного случая) при векторном аргументе.

Значение параметра $z_k$ мы моделируем линейной функцией от признаков объекта (размерности M) и параметров модели для класса $k$:

$$
z_k = \sum_{i=1}^Mw_{ki}x_i
$$

---

Для моделирования искомого распределения будем использовать __[категориальное распределение](https://en.wikipedia.org/wiki/Categorical_distribution)__. Запишем функцию правдоподобия:

$$
L\left(\textbf{y} \mid \textbf{p}\right) = 
\prod_{i=1}^{K}p_{i}^{y_{i}} = 
\prod_{i=1}^{K}\sigma_{i}(\textbf{z})^{y_{i}}
$$

Поскольку логарифм положительного аргумента монотонно возрастает на всей области определения, то логарифмирование функции правдоподобия не изменит положение ее максимума. Значит, для удобства мы можем воспользоваться логарифмом функции правдоподобия:

$$
\mathcal{L} = \log L = 
\log \left(\prod_{i=1}^{K}\sigma_{i}(\textbf{z})^{y_{i}}\right) = 
\sum_{i=1}^{K}y_{i}\log \sigma_{i}(\textbf{z}) \rightarrow max
$$

Если домножить на $(-1)$, то получится выражение для __[кросс-энтропии](https://en.wikipedia.org/wiki/Cross_entropy)__ между искомым распределением и распределением оценок обучающей выборки. Правдоподобие мы максимизируем, а кросс-энтропию, соответственно, минимизируем.

$$
H = \left(-\mathcal{L}\right) \rightarrow min
$$

Для этого будем использовать методы градиентного спуска. Необходимо вывести выражение для компонент вектора градиента кросс-энтропии:

$$
\dfrac {\partial H} {\partial w_{km}}
$$

Рассмотрим следующие частные производные:

$$
\dfrac {\partial H} {\partial \sigma_{k}} =
\dfrac {\partial} {\partial \sigma_{k}} \left( -\sum_{i=1}^{K} y_i \log \sigma_i \right) =
- \dfrac {y_k} {\sigma_k}
$$
<br><br>
$$
\dfrac {\partial \sigma_i} {\partial z_k} = 
\dfrac {\partial} {\partial z_k} \left( \dfrac {e^{z_i}} {\sum_{j=1}^{K} e^{z_j}} \right) =
\left \{ \begin{array}{lcl} 
\dfrac {1} {\left( \sum_{j=1}^{K}e^{z_j} \right)^2 } \left( e^{z_k} \sum_{j=1}^{K}e^{z_j} - e^{z_k} \cdot e^{z_k} \right) &=&
\sigma_k \left( 1 - \sigma_k \right) && (i=k)
\\ 
\dfrac {1} {\left( \sum_{j=1}^{K}e^{z_j} \right)^2 } \left( -e^{z_i} \cdot e^{z_k} \right) &=&
- \sigma_i \sigma_k && (i \neq k)
\end{array} \right.
$$
<br><br>
$$
\dfrac {\partial z_k} {\partial w_{km}} =
\dfrac {\partial} {\partial w_{km}} \left( \sum_{j=1}^{M} w_{kj}x_j \right) = x_m
$$

Теперь мы можем записать:

$$
\dfrac {\partial H} {\partial z_k} = 
\sum_{i=1}^{K} \dfrac {\partial H} {\partial \sigma_i} \dfrac {\partial \sigma_i} {\partial z_k} =
\dfrac {\partial H} {\partial \sigma_k} \dfrac {\partial \sigma_k} {\partial z_k} +
\sum_{i \neq k} \dfrac {\partial H} {\partial \sigma_i} \dfrac {\partial \sigma_i} {\partial z_k} =
-y_k \left( 1- \sigma_k \right) + \sum_{i \neq k} y_i \sigma_k =
-y_k + \sigma_k \sum_{i} y_i =
\sigma_k - y_k
$$
<br><br>
$$
\dfrac {\partial H} {\partial w_{km}} =
\sum_{i=1}^{K} \dfrac {\partial H} {\partial z_i} \dfrac {\partial z_i} {\partial w_{km}} =
x_m \left( \sigma_k - y_k \right)
$$